In [74]:
import os
from django.core.management import call_command

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [87]:
import re
from hl7_app.models import Patient, ErrorCharges
from datetime import datetime


In [93]:

# Function to read HL7 file
def read_hl7_file(file_path):
    with open(file_path, 'r') as file:
        hl7_data = file.read()
    return hl7_data

# Function to extract patient data from HL7 text
def extract_patient_data(hl7_data):
    patients = []

    # Split the HL7 data into segments
    segments = hl7_data.split('\n')

    for segment in segments:
        if segment.startswith('PID'):
            # Extract fields from PID segment
            fields = segment.split('|')
            try:
                patient_data = {
                    'mrn': fields[3].split('^')[0],  # Medical Record Number
                    'last_name': fields[5].split('^')[0],  # Last Name
                    'first_name': fields[5].split('^')[1] if len(fields[5].split('^')) > 1 else 'N/A',  # First Name
                    'date_of_birth': fields[7],  # Date of Birth
                    'phone': fields[11].split('^')[0]  # Phone
                }
                patients.append(patient_data)
            except IndexError:
                # Handle unexpected format
                print("Error processing PID segment:", segment)
    
    return patients


def store_patient_data(patient_data):
    print(patient_data)
    for entry in patient_data:
        first_name = entry.get('first_name')
        last_name = entry.get('last_name')
        date_of_birth_str = entry.get('date_of_birth')
        mrn = entry.get('mrn')
        phone = entry.get('phone')
        print(first_name,last_name, mrn, type(mrn))
        errors = []
        if not date_of_birth_str:
            errors.append('DOB not found')
        if not mrn:
            errors.append('MRN not found')
        if not first_name or not last_name:
            errors.append('Patient name not found')
    
        try:
            date_of_birth = datetime.strptime(date_of_birth_str, '%Y%m%d').date()
        except ValueError:
            date_of_birth = None
            errors.append('Invalid DOB format')
    
        if not errors:
            existing_patients = Patient.objects.filter(mrn=mrn)
            if existing_patients.exists():
                # Handle duplicates - choose one or merge
                for patient in existing_patients:
                    # For demonstration, we'll just print duplicates
                    print(f"Duplicate MRN found: {patient.mrn}")
            else:
                Patient.objects.create(
                    mrn=mrn,
                    first_name=first_name,
                    last_name=last_name,
                    date_of_birth=date_of_birth,
                    phone=phone
                )
        else:
            for error in errors:
                print(errors)
                print(error)
                print(mrn)
                ErrorCharges.objects.create(
                    error_type=error,
                    error_message=f"Missing data for MRN: {mrn}",
                   
                )


In [96]:
# Path to HL7 file
file_path = r'C:\Users\91824\Downloads\HL7 (1) (1)\HL7\charges_0.hl7'

# Read HL7 data
hl7_data = read_hl7_file(file_path)

# Extract patient data
patient_data = extract_patient_data(hl7_data)
store_patient_data(patient_data)

# Display extracted data
print(patient_data)


[{'mrn': '789101', 'last_name': 'Smith', 'first_name': 'Jane', 'date_of_birth': '19920322', 'phone': '456 Oak St'}, {'mrn': '112233', 'last_name': '', 'first_name': '', 'date_of_birth': '19850110', 'phone': '789 Pine St'}, {'mrn': '334455', 'last_name': 'Brown', 'first_name': 'Olivia', 'date_of_birth': '19951005', 'phone': '321 Cedar St'}, {'mrn': '556677', 'last_name': 'White', 'first_name': 'Emily', 'date_of_birth': '20000515', 'phone': '654 Spruce St'}, {'mrn': '778899', 'last_name': 'Wilson', 'first_name': 'James', 'date_of_birth': '19780708', 'phone': '987 Birch St'}, {'mrn': '990011', 'last_name': 'Garcia', 'first_name': 'Sophia', 'date_of_birth': '19990214', 'phone': '234 Willow St'}, {'mrn': '001122', 'last_name': 'Martinez', 'first_name': 'Daniel', 'date_of_birth': '19730518', 'phone': '567 Aspen St'}, {'mrn': '223344', 'last_name': 'Lee', 'first_name': 'Chloe', 'date_of_birth': '20010323', 'phone': '890 Maple St'}, {'mrn': '334455', 'last_name': 'Kim', 'first_name': 'Ethan', 

OperationalError: (1054, "Unknown column 'date_occurred' in 'field list'")